<a href="https://colab.research.google.com/github/vaishnavikale404/DataScienceLab_64/blob/main/exp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E']
}

print("Graph definition (adjacency list):")
for node, neighbors in graph.items():
    print(f"  {node}: {neighbors}")

Graph definition (adjacency list):
  A: ['B', 'C']
  B: ['A', 'D', 'E']
  C: ['A', 'F']
  D: ['B']
  E: ['B', 'F']
  F: ['C', 'E']


### Breadth-First Search (BFS)

BFS explores a graph layer by layer, starting from a given source node. It visits all the neighbors of the current node before moving on to their neighbors. Think of it like ripples in a pond spreading outwards.

**How it works:**
1.  Start at a chosen node (the 'root').
2.  Visit all its immediate neighbors.
3.  Then, visit all the neighbors of those neighbors.
4.  Continue this process until all reachable nodes have been visited.

It typically uses a **queue** data structure to keep track of nodes to visit next.

In [3]:
from collections import deque

def bfs(graph, start_node):
    visited = set()  # To keep track of visited nodes
    queue = deque([start_node])  # Queue for nodes to visit
    traversal_path = [] # To store the order of visited nodes

    visited.add(start_node)

    while queue:
        current_node = queue.popleft() # Get the next node from the queue
        traversal_path.append(current_node)

        for neighbor in graph[current_node]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor) # Add unvisited neighbors to the queue
    return traversal_path

print("BFS Traversal starting from 'A':")
print(bfs(graph, 'A'))

BFS Traversal starting from 'A':
['A', 'B', 'C', 'D', 'E', 'F']


### Depth-First Search (DFS)

DFS explores a graph by going as deep as possible along each branch before backtracking. Imagine exploring a maze: you go down one path until you hit a dead end, then backtrack and try another path.

**How it works:**
1.  Start at a chosen node.
2.  Go as deep as possible along one path until you reach a node with no unvisited neighbors (or a dead end).
3.  Backtrack to the previous node and explore another path from there.
4.  Repeat until all reachable nodes are visited.

It typically uses a **stack** data structure (or recursion, which uses the call stack implicitly) to keep track of nodes to visit next.

In [4]:
def dfs(graph, start_node):
    visited = set()  # To keep track of visited nodes
    stack = [start_node]  # Stack for nodes to visit
    traversal_path = [] # To store the order of visited nodes

    while stack:
        current_node = stack.pop() # Get the next node from the stack

        if current_node not in visited:
            visited.add(current_node)
            traversal_path.append(current_node)

            # Add neighbors to the stack (order might affect path, often reversed for consistent output)
            # Reversing ensures that if A has B, C as neighbors, and we add C then B, B is processed first (LIFO)
            for neighbor in reversed(graph[current_node]):
                if neighbor not in visited:
                    stack.append(neighbor)
    return traversal_path

print("DFS Traversal starting from 'A':")
print(dfs(graph, 'A'))

DFS Traversal starting from 'A':
['A', 'B', 'D', 'E', 'F', 'C']


### A* Search Algorithm

A* (pronounced "A-star") is a popular pathfinding algorithm, widely used in areas like game development and robotics. It's essentially an improvement over Dijkstra's algorithm because it uses a 'heuristic' to guide its search.

Think of it like planning a road trip: Dijkstra's would find the shortest path by exploring all possible roads in increasing order of distance from your start. A* is smarter: it prioritizes exploring roads that seem to be heading in the general direction of your destination.

**How it works:**

A* finds the path from a starting node to a goal node with the smallest cost. It does this by evaluating each node `n` using a special function `f(n)`:

`f(n) = g(n) + h(n)`

*   `g(n)`: This is the actual cost from the starting node to node `n`. (What you've already paid to get here).
*   `h(n)`: This is the heuristic estimate of the cost from node `n` to the goal node. (An educated guess of how much more you'll have to pay).

A* always expands the node with the lowest `f(n)` value. The heuristic `h(n)` must be admissible (never overestimates the true cost to the goal) to guarantee finding the shortest path.

**Common Use Cases:**
*   Finding the shortest path in video games.
*   Robot navigation.
*   Solving puzzles like the 8-puzzle.

### Simple Grid-Based Problem for A*

Let's consider a simple grid (like a maze) where:
*   `S` is the Start point.
*   `G` is the Goal point.
*   `#` represents an obstacle (wall).
*   `.` represents an open path.

We want to find the shortest path from `S` to `G`, moving only horizontally or vertically (no diagonal moves). Each move to an adjacent open cell has a cost of 1.

**Heuristic:** For this grid, a good heuristic `h(n)` is the **Manhattan distance** (or taxicab geometry) to the goal. This is the sum of the absolute differences of their coordinates: `abs(x1 - x2) + abs(y1 - y2)`.

In [6]:
import heapq

# Define the grid (S=Start, G=Goal, #=Wall, .=Path)
grid = [
    list("S........"),
    list("###.#####"),
    list("...G#####"),
    list("#.#######"),
    list("#.#######"),
    list("#........")
]

# Find start and goal coordinates
start = None
goal = None
for r_idx, row in enumerate(grid):
    for c_idx, cell in enumerate(row):
        if cell == 'S':
            start = (r_idx, c_idx)
        elif cell == 'G':
            goal = (r_idx, c_idx)

if not start or not goal:
    raise ValueError("Start 'S' or Goal 'G' not found in the grid.")

rows = len(grid)
cols = len(grid[0])

print("Grid representation:")
for row in grid:
    print(" ".join(row))
print(f"Start: {start}, Goal: {goal}")

Grid representation:
S . . . . . . . .
# # # . # # # # #
. . . G # # # # #
# . # # # # # # #
# . # # # # # # #
# . . . . . . . .
Start: (0, 0), Goal: (2, 3)


In [7]:
def manhattan_distance(pos1, pos2):
    """Calculates the Manhattan distance between two points."""
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

def a_star(grid, start, goal):
    rows, cols = len(grid), len(grid[0])
    # priority_queue: stores (f_score, g_score, current_pos, path)
    # f_score = g_score + h_score
    # g_score = actual cost from start to current_pos
    # path = list of coordinates leading to current_pos
    pq = [(manhattan_distance(start, goal), 0, start, [start])]

    # visited_g_scores: stores the lowest g_score found to reach a cell
    visited_g_scores = {start: 0}

    while pq:
        f_score, g_score, current_pos, path = heapq.heappop(pq)

        # If we reached the goal, return the path
        if current_pos == goal:
            return path

        # Explore neighbors
        r, c = current_pos
        # Possible movements: up, down, left, right
        for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            neighbor_r, neighbor_c = r + dr, c + dc
            neighbor_pos = (neighbor_r, neighbor_c)

            # Check if neighbor is within grid bounds and not an obstacle
            if 0 <= neighbor_r < rows and 0 <= neighbor_c < cols and grid[neighbor_r][neighbor_c] != '#':
                # Cost to move to neighbor is 1
                new_g_score = g_score + 1

                # If this is a better path to the neighbor, update it
                if neighbor_pos not in visited_g_scores or new_g_score < visited_g_scores[neighbor_pos]:
                    visited_g_scores[neighbor_pos] = new_g_score
                    h_score = manhattan_distance(neighbor_pos, goal)
                    new_f_score = new_g_score + h_score
                    new_path = path + [neighbor_pos]
                    heapq.heappush(pq, (new_f_score, new_g_score, neighbor_pos, new_path))

    return None # No path found

print("Running A* search...")
path = a_star(grid, start, goal)

if path:
    print("Path found:")
    # Create a visual representation of the path
    path_grid = [row[:] for row in grid] # Make a copy of the grid
    for r, c in path:
        if (r, c) != start and (r, c) != goal:
            path_grid[r][c] = '*'

    for row in path_grid:
        print(" ".join(row))
    print(f"Path length: {len(path) - 1} steps") # Subtract 1 for start node
else:
    print("No path found from Start to Goal.")

Running A* search...
Path found:
S * * * . . . . .
# # # * # # # # #
. . . G # # # # #
# . # # # # # # #
# . # # # # # # #
# . . . . . . . .
Path length: 5 steps


### Minimax Algorithm: Thinking Ahead in Games

Minimax is a decision-making algorithm, typically used in game theory and artificial intelligence, for two-player turn-based games (like Tic-Tac-Toe or Chess). It helps an AI player choose the best move possible by considering all possible moves the opponent could make.

**The Core Idea:**
Imagine you're playing a game, and it's your turn. You want to make a move that leads to the best possible outcome for *you*. But your opponent is also smart and will try to make the best possible outcome for *them* (which is usually the worst for you!).

Minimax works by:
1.  **Maximizing Player (You/AI):** Tries to get the highest possible score.
2.  **Minimizing Player (Opponent):** Tries to force the lowest possible score on you.

The algorithm explores the 'game tree' (all possible future moves and their outcomes) until it reaches an end state of the game (win, loss, or draw). It then works backward from these end states, assigning scores to each move, assuming both players play optimally.

**Let's use a simple Tic-Tac-Toe game to demonstrate this!**

*   **Our AI will be 'X' (the Maximizing player).** It wants to win.
*   **The opponent will be 'O' (the Minimizing player).** It wants to make 'X' lose or draw.
*   **Scores:**
    *   +10 for 'X' win
    *   -10 for 'O' win
    *   0 for a draw

In [8]:
# Helper functions for a simple Tic-Tac-Toe game

def print_board(board):
    """Prints the Tic-Tac-Toe board."""
    for row in board:
        print(" | ".join(row))
        print("---------")

def check_winner(board, player):
    """Checks if a player has won."""
    # Check rows
    for row in board:
        if all([s == player for s in row]):
            return True
    # Check columns
    for col in range(3):
        if all([board[row][col] == player for row in range(3)]):
            return True
    # Check diagonals
    if all([board[i][i] == player for i in range(3)]) or \
       all([board[i][2 - i] == player for i in range(3)]):
        return True
    return False

def is_board_full(board):
    """Checks if the board is full (a draw if no winner)."""
    for row in board:
        for cell in row:
            if cell == ' ':
                return False
    return True

def get_empty_cells(board):
    """Returns a list of (row, col) for empty cells."""
    empty_cells = []
    for r in range(3):
        for c in range(3):
            if board[r][c] == ' ':
                empty_cells.append((r, c))
    return empty_cells

def evaluate_board(board):
    """Evaluates the board state: +10 for X win, -10 for O win, 0 for draw, None if game not over."""
    if check_winner(board, 'X'):
        return 10
    elif check_winner(board, 'O'):
        return -10
    elif is_board_full(board):
        return 0
    return None # Game not over

print("Game setup functions are ready!")

Game setup functions are ready!


In [9]:
def minimax(board, depth, is_maximizing):
    """The Minimax algorithm function."""
    # Base cases: if the game is over, return the score
    score = evaluate_board(board)
    if score is not None:
        return score

    # If it's the Maximizing player's turn ('X')
    if is_maximizing:
        best_score = -float('inf') # Start with a very low score
        for r, c in get_empty_cells(board):
            # Make the move
            board[r][c] = 'X'
            # Recursively call minimax for the next turn (Minimizing player)
            current_score = minimax(board, depth + 1, False)
            # Undo the move (important for exploring other paths)
            board[r][c] = ' '
            # Update best_score if this path is better
            best_score = max(best_score, current_score)
        return best_score

    # If it's the Minimizing player's turn ('O')
    else:
        best_score = float('inf') # Start with a very high score
        for r, c in get_empty_cells(board):
            # Make the move
            board[r][c] = 'O'
            # Recursively call minimax for the next turn (Maximizing player)
            current_score = minimax(board, depth + 1, True)
            # Undo the move
            board[r][c] = ' '
            # Update best_score if this path is better
            best_score = min(best_score, current_score)
        return best_score

def find_best_move(board):
    """Finds the optimal move for the AI ('X') using Minimax."""
    best_val = -float('inf')
    best_move = None

    for r, c in get_empty_cells(board):
        # Try this move
        board[r][c] = 'X'
        # Calculate the value of this move using minimax (assuming opponent plays optimally)
        move_val = minimax(board, 0, False) # After AI's move, it's opponent's turn (Minimizing)
        # Undo the move
        board[r][c] = ' '

        # If this move leads to a better outcome, remember it
        if move_val > best_val:
            best_val = move_val
            best_move = (r, c)

    return best_move

print("Minimax algorithm and best move finder are implemented!")

Minimax algorithm and best move finder are implemented!


In [10]:
print("--- Minimax Demonstration (AI is 'X') ---")

# Scenario 1: Empty board
current_board_1 = [
    [' ', ' ', ' '],
    [' ', ' ', ' '],
    [' ', ' ', ' ']
]
print("\nInitial Board (Empty):")
print_board(current_board_1)
best_move_1 = find_best_move(current_board_1)
print(f"AI's Best Move: {best_move_1}")
if best_move_1:
    current_board_1[best_move_1[0]][best_move_1[1]] = 'X'
    print("Board after AI's move:")
    print_board(current_board_1)

# Scenario 2: A mid-game state where AI can win
current_board_2 = [
    ['X', 'O', ' '],
    [' ', 'X', ' '],
    ['O', ' ', ' ']
]
print("\nInitial Board (Mid-game, AI can win):")
print_board(current_board_2)
best_move_2 = find_best_move(current_board_2)
print(f"AI's Best Move: {best_move_2}")
if best_move_2:
    current_board_2[best_move_2[0]][best_move_2[1]] = 'X'
    print("Board after AI's move:")
    print_board(current_board_2)

# Scenario 3: A mid-game state where AI needs to block opponent
current_board_3 = [
    ['X', 'O', ' '],
    [' ', ' ', ' '],
    ['O', ' ', 'X']
]
print("\nInitial Board (Mid-game, AI needs to block):")
print_board(current_board_3)
best_move_3 = find_best_move(current_board_3)
print(f"AI's Best Move: {best_move_3}")
if best_move_3:
    current_board_3[best_move_3[0]][best_move_3[1]] = 'X'
    print("Board after AI's move:")
    print_board(current_board_3)

--- Minimax Demonstration (AI is 'X') ---

Initial Board (Empty):
  |   |  
---------
  |   |  
---------
  |   |  
---------
AI's Best Move: (0, 0)
Board after AI's move:
X |   |  
---------
  |   |  
---------
  |   |  
---------

Initial Board (Mid-game, AI can win):
X | O |  
---------
  | X |  
---------
O |   |  
---------
AI's Best Move: (1, 0)
Board after AI's move:
X | O |  
---------
X | X |  
---------
O |   |  
---------

Initial Board (Mid-game, AI needs to block):
X | O |  
---------
  |   |  
---------
O |   | X
---------
AI's Best Move: (0, 2)
Board after AI's move:
X | O | X
---------
  |   |  
---------
O |   | X
---------
